In [3]:
# Now I want write a test script to test all testcases in testcases folder
# The testcases in such format
# <name>.sy source code
# <name>.in input
# <name>.out std output

# compiler path: ./build/SYSY-compiler
# compiler command: ./build/SYSY-compiler <name>.sy
# I need you to run the compiler and get the output llvm ir code, which will be put in <name>.ll 
# If the compiler doesn't end with 0, keep a record of the trouble file name in CE_list
# If the compiler ends with 0, run the llvm ir code with lli and get the output, which will be put in <name>.outq
# Compare it with the std output <name>.out ,if is't not that same ,keep a record of the trouble file name in WA_list
# Remember to redirect the input and output of lli

import os
import sys
import subprocess
import filecmp
import difflib

compiler_path = "./build/SYSY-compiler"
test_folder="./testcases"

pass_args=["--mem2reg","--dce","--cse"]

# grab all the testcases

# find files recursively
test_list = []
for root, dirs, files in os.walk(test_folder):
    for file in files:
        if file.endswith(".sy"):
            test_list.append(os.path.join(root, file))

CE_list = []
LLI_FAIL_list = []
WA_list = []
AC_list = []
TLE_list = []
Time_Out = []

Bad_test = []

for test in test_list:
    if test.endswith(".sy"):
        # add test_args to list
        compile_args=[compiler_path, test]
        for arg in pass_args:
            compile_args.append(arg)
        try:
            ret = subprocess.run(compile_args,timeout=60)
        except subprocess.TimeoutExpired:
            Time_Out.append(test)
            continue
        if ret.returncode != 0:
            CE_list.append(test)
            print("Compiler Error："+test)
            continue
        # run lli
        ll_file = test+".ll"
        out_file = test[:-2]+"out"
        # riscv file
        # asm_file = test + "..s"
        # maybe theres no input file
        
        if not os.path.exists(test[:-2]+"in") and not os.path.exists(out_file):
            Bad_test.append(test)
            continue
        
        # Kill program run over 5s, give it a TLE
        try:
            if not os.path.exists(test[:-2]+"in"):
                ret = subprocess.run(["lli-14","-opaque-pointers",ll_file], stdout=subprocess.PIPE, stderr=subprocess.PIPE, timeout=10)
                # ret1 = subprocess.run(["riscv64-unknown-elf-as","-o","output.o",asm_file], stdout=subprocess.PIPE, stderr=subprocess.PIPE, timeout=10)
            else:
                ret = subprocess.run(["lli-14","-opaque-pointers",ll_file], stdout=subprocess.PIPE, stderr=subprocess.PIPE, input=open(test[:-2]+"in").read().encode(), timeout=10)
                # ret1 = subprocess.run(["riscv64-unknown-elf-as","-o","output.o",asm_file], stdout=subprocess.PIPE, stderr=subprocess.PIPE, timeout=10)
        except subprocess.TimeoutExpired:
            print("Timeout Error")
            TLE_list.append(test)
            print(test)
            continue
            
        # compare the output
        # Merge The reture Value and stdout
        
        dump_str=ret.stdout.decode()
        # dump_str1 = ret1.stdout.decode()
        # remove whitesspace in the end
        # dump_str=dump_str.rstrip()
        # if dump_str1 and not dump_str1.endswith('\n'):
        #     dump_str1 += "\n"
        # if not dump_str1.endswith(''):
        #     print("RISCV Test Error")
        if dump_str and not dump_str.endswith('\n'):
            dump_str += "\n"
        dump_str += str(ret.returncode) + "\n"
        std_output=open(out_file).read()
        diff = difflib.unified_diff(dump_str.splitlines(), std_output.splitlines(), lineterm='')
        if(len(list(diff))!=0):
            print("Wrong Answer")
            WA_list.append(test)
            print(test)
            continue
        # print("Accepted")
        AC_list.append(test)
    
        
print("Compiler Error: Total: "+str(len(CE_list)))
print("Runtime Error: Total: "+str(len(LLI_FAIL_list)))
print("Timeout Error: Total: "+str(len(TLE_list)))
print("Wrong Answer: Total: "+str(len(WA_list)))
print("TimeOut Function:"+str(len(Time_Out)))
print("Bad Test: Total: "+str(len(Bad_test)))
print("Accepted: Total: "+str(len(AC_list)))

Function: myabs CSE: 0
Function: func CSE: 0
Function: main CSE: 0
Function: main CSE: 0
Function: getMaxNum CSE: 0
Function: getNumPos CSE: 0
Function: radixSort CSE: 2
Function: main CSE: 2


Wrong Answer
./testcases/final_performance/03_sort1.sy


Function: multiply CSE: 0
Function: power CSE: 0
Function: memmove CSE: 0
Function: fft CSE: 0
Function: main CSE: 12


Wrong Answer
./testcases/final_performance/fft0.sy


Function: myabs CSE: 0
Function: func CSE: 0
Function: main CSE: 0
Function: hash CSE: 0
Function: insert CSE: 10
Function: reduce CSE: 0
Function: main CSE: 0
Function: main CSE: 0
Function: read_map CSE: 0
Function: put_map CSE: 0
Function: swap12 CSE: 0
Function: step CSE: 0
Function: main CSE: 0
Function: multiply CSE: 0
Function: power CSE: 0
Function: memmove CSE: 0
Function: fft CSE: 0
Function: main CSE: 12


...

Function: func CSE: 0
Function: main CSE: 0
Function: get_random CSE: 6
Function: rotl1 CSE: 0
Function: rotl5 CSE: 0
Function: rotl30 CSE: 0
Function: _and CSE: 0
Function: _not CSE: 0
Function: _xor CSE: 0
Function: _or CSE: 0
Function: pseudo_sha1 CSE: 0
Function: main CSE: 0
Function: get_random CSE: 6
Function: rotl1 CSE: 0
Function: rotl5 CSE: 0
Function: rotl30 CSE: 0
Function: _and CSE: 0
Function: _not CSE: 0
Function: _xor CSE: 0
Function: _or CSE: 0
Function: pseudo_sha1 CSE: 0
Function: main CSE: 0
Function: transpose CSE: 0
Function: main CSE: 0
Function: mm CSE: 0
Function: main CSE: 0
Function: func CSE: 0
Function: main CSE: 0
Function: read_map CSE: 0
Function: put_map CSE: 0
Function: swap12 CSE: 0
Function: step CSE: 0
Function: main CSE: 0
Function: spmv CSE: 0
Function: main CSE: 0
Function: cutout CSE: 0
Function: main CSE: 0
Function: meanless_calculation CSE: 0
Function: swap CSE: 2
Function: median CSE: 0
Function: main CSE: 5
Function: mm CSE: 0
Function: main

Timeout Error
./testcases/performance/hoist-2.sy


Function: func CSE: 0
Function: main CSE: 0
Function: transpose CSE: 0
Function: main CSE: 0
Function: func CSE: 0
Function: main CSE: 0
Function: getMaxNum CSE: 0
Function: getNumPos CSE: 0
Function: radixSort CSE: 2
Function: main CSE: 2
Function: checkrange CSE: 0
Function: reduce CSE: 0
Function: getvalue CSE: 0
Function: convn CSE: 0
Function: memmove CSE: 0
Function: main CSE: 1


Wrong Answer
./testcases/performance/03_sort2.sy
Wrong Answer
./testcases/performance/00_bitset2.sy
Compiler Error: Total: 0
Runtime Error: Total: 0
Timeout Error: Total: 3
Wrong Answer: Total: 19
TimeOut Function:0
Bad Test: Total: 0
Accepted: Total: 213


Function: set CSE: 0
Function: rand CSE: 3
Function: main CSE: 0
Function: getvalue CSE: 0
Function: mem_move CSE: 0
Function: floyd CSE: 0
Function: main CSE: 2


In [ ]:
import os
import sys
import subprocess
import filecmp
import difflib

test_folder="./testcase"

# grab all the testcases

# find files recursively
test_list = []
for root, dirs, files in os.walk(test_folder):
    for file in files:
        if file.endswith(".s"):
            test_list.append(os.path.join(root, file))

CE_list = []
AC_list = []

for test in test_list:
    if test.endswith(".s"):
        # add test_args to list
        compile_args=["riscv64-unknown-elf-as", test, "-o", "a.out"]
        try:
            ret = subprocess.run(compile_args,timeout=60)
        except subprocess.TimeoutExpired:
            Time_Out.append(test)
            continue
        if ret.returncode != 0:
            CE_list.append(test)
            print("Compiler Error："+test)
            continue
        AC_list.append(test)
    
        
print("Compiler Error: Total: "+str(len(CE_list)))
print("Accepted: Total: "+str(len(AC_list)))

In [ ]:
# Now I want write a test script to test all testcases in testcases folder
# The testcases in such format
# <name>.sy source code
# <name>.in input
# <name>.out std output

# compiler path: ./build/SYSY-compiler
# compiler command: ./build/SYSY-compiler <name>.sy
# I need you to run the compiler and get the output llvm ir code, which will be put in <name>.ll 
# If the compiler doesn't end with 0, keep a record of the trouble file name in CE_list
# If the compiler ends with 0, run the llvm ir code with lli and get the output, which will be put in <name>.outq
# Compare it with the std output <name>.out ,if is't not that same ,keep a record of the trouble file name in WA_list
# Remember to redirect the input and output of lli

import os
import sys
import subprocess
import filecmp
import difflib

compiler_path = "./build/SYSY-compiler"
test_folder="./testcases"

pass_args=["--mem2reg","--constprop","--reassociate","--simplifycfg"]

# grab all the testcases

# find files recursively
test_list = []
for root, dirs, files in os.walk(test_folder):
    for file in files:
        if file.endswith(".sy"):
            test_list.append(os.path.join(root, file))

Middle_CE_list = []
LLI_FAIL_list = []
Middle_WA_list = []
Middle_AC_list = []
Middle_TLE_list = []
Time_Out = []

Bad_test = []

Backend_WA_list = []
Backend_CE_list = []
Backend_AC_list = []
Backend_TLE_list = []
for test in test_list:
    if test.endswith(".sy"):
        # add test_args to list
        compile_args=[compiler_path, test]
        for arg in pass_args:
            compile_args.append(arg)
        try:
            ret = subprocess.run(compile_args,timeout=60)
        except subprocess.TimeoutExpired:
            Time_Out.append(test)
            continue
        if ret.returncode != 0:
            Middle_CE_list.append(test)
            print("Compiler Error："+test)
            continue
        # run lli
        ll_file = test+".ll"
        out_file = test[:-2]+"out"

        # maybe theres no input file
        
        if not os.path.exists(test[:-2]+"in") and not os.path.exists(out_file):
            Bad_test.append(test)
            continue
        
        # Kill program run over 5s, give it a TLE
        try:
            if not os.path.exists(test[:-2]+"in"):
                ret = subprocess.run(["lli-14","-opaque-pointers",ll_file], stdout=subprocess.PIPE, stderr=subprocess.PIPE, timeout=10)
                # ret1 = subprocess.run(["riscv64-unknown-elf-as","-o","output.o",asm_file], stdout=subprocess.PIPE, stderr=subprocess.PIPE, timeout=10)
            else:
                ret = subprocess.run(["lli-14","-opaque-pointers",ll_file], stdout=subprocess.PIPE, stderr=subprocess.PIPE, input=open(test[:-2]+"in").read().encode(), timeout=10)
                # ret1 = subprocess.run(["riscv64-unknown-elf-as","-o","output.o",asm_file], stdout=subprocess.PIPE, stderr=subprocess.PIPE, timeout=10)
        except subprocess.TimeoutExpired:
            print("Middle Timeout Error")
            Middle_TLE_list.append(test)
            print(test)
            continue
            
        # compare the output
        # Merge The reture Value and stdout
        
        dump_str=ret.stdout.decode()
        # dump_str1 = ret1.stdout.decode()
        # remove whitesspace in the end
        # dump_str=dump_str.rstrip()
        # if dump_str1 and not dump_str1.endswith('\n'):
        #     dump_str1 += "\n"
        # if not dump_str1.endswith(''):
        #     print("RISCV Test Error")
        if dump_str and not dump_str.endswith('\n'):
            dump_str += "\n"
        dump_str += str(ret.returncode) + "\n"
        std_output=open(out_file).read()
        diff = difflib.unified_diff(dump_str.splitlines(), std_output.splitlines(), lineterm='')
        if(len(list(diff))!=0):
            print("Middle Wrong Answer")
            Middle_WA_list.append(test)
            continue
        # print("Accepted")
        Middle_AC_list.append(test)


        # riscv file
        asm_file = test + "..s"
        try:
            if not os.path.exists(test[:-2]+"in"):
                subprocess.run(["gcc","-march=rv64gc","-o","output.o",asm_file,"-lsyslib"], timeout=10)
                ret1 = subprocess.run(["./ouput.o"], stdout=subprocess.PIPE, stderr=subprocess.PIPE, timeout = 60)
            else:
                subprocess.run(["gcc","-march=rv64gc","-o","output.o",asm_file, "-lsyslib"], stdout=subprocess.PIPE, stderr=subprocess.PIPE, input=open(test[:-2]+"in").read().encode(), timeout=10)
                ret1 = subprocess.run(["./output.o"], stdout=subprocess.PIPE, stderr=subprocess.PIPE, input=open(test[:-2]+"in").read().encode(), timeout=60)
        except subprocess.TimeoutExpired:
            print("Backend Timeout Error")
            Backend_TLE_list.append(test)
            print(test)
            continue
            
        # compare the output
        # Merge The reture Value and stdout
        
        dump_str1=ret1.stdout.decode()
        if dump_str1 and not dump_str1.endswith('\n'):
            dump_str1 += "\n"
        dump_str1 += str(ret1.returncode) + "\n"
        std_output1=open(out_file).read()
        diff1 = difflib.unified_diff(dump_str1.splitlines(), std_output1.splitlines(), lineterm='')
        if(len(list(diff1))!=0):
            print("Backend Wrong Answer")
            Backend_WA_list.append(test)
            continue
        # print("Accepted")
        Middle_AC_list.append(test)

print("TimeOut Function:"+str(len(Time_Out)))
print("Bad Test: Total: "+str(len(Bad_test)))
print("Runtime Error: Total: "+str(len(LLI_FAIL_list)))
print("Middle Compiler Error: Total: "+str(len(Middle_CE_list)))
print("Middle Timeout Error: Total: "+str(len(Middle_TLE_list)))
print("Middle Wrong Answer: Total: "+str(len(Middle_WA_list)))
print("Middle Accepted: Total: "+str(len(Middle_AC_list)))

print("Backend Compiler Error: Total: "+str(len(Backend_CE_list)))
print("Backend Timeout Error: Total: "+str(len(Backend_TLE_list)))
print("Backend Wrong Answer: Total: "+str(len(Backend_WA_list)))
print("Backend Accepted: Total: "+str(len(Backend_AC_list)))


In [3]:
for k in range(len(WA_list)):
    if len(WA_list) > k:
        print(WA_list[k])

./testcases/hidden_functional/18_prim.sy


In [ ]:
import os
import sys
import subprocess
import filecmp
import difflib

test_folder="./testcases"

# grab all the testcases

# find files recursively
test_list = []
for root, dirs, files in os.walk(test_folder):
    for file in files:
        if file.endswith(".s"):
            test_list.append(os.path.join(root, file))

CE_list = []
AC_list = []

for test in test_list:
    if test.endswith(".s"):
            os.remove(test)

In [2]:
for k in range(len(WA_list)):
    if len(WA_list) > k:
        print(WA_list[k])

./testcases/performance/median2.sy


In [ ]:
for k in range(len(CE_list)):
    if len(CE_list) > k:
        print(CE_list[k])

In [ ]:
for k in range(len(TLE_list)):
    if len(TLE_list) > k:
        print(TLE_list[k])

In [ ]:
import os
import shutil

current_dir = os.getcwd()
target_folder=os.path.join(current_dir,"include")
# 遍历当前目录下的所有文件和文件夹
for root, dirs, files in os.walk(current_dir):
    if root=='/mnt/e/miniC-compiler/ir/opt':
        os.makedirs(os.path.join(target_folder,"ir/opt"),exist_ok=True)
        for file in files:
            if file.endswith(".hpp"):
                if not os.path.exists(os.path.join(target_folder,"ir/opt")+file):
                    path=os.path.join(root,file)
                    shutil.move(path,os.path.join(target_folder,"ir/opt"))
    if root=='/mnt/e/miniC-compiler/backend':
        os.makedirs(os.path.join(target_folder,"backend"),exist_ok=True)
        for file in files:
            if file.endswith(".hpp"):
                if not os.path.exists(os.path.join(target_folder,"backend")+file):
                    path=os.path.join(root,file)
                    shutil.move(path,os.path.join(target_folder,"backend"))
    if root=='/mnt/e/miniC-compiler/ir/Analysis':
        os.makedirs(os.path.join(target_folder,"ir/Analysis"),exist_ok=True)
        for file in files:
            if file.endswith(".hpp"):
                if not os.path.exists(os.path.join(target_folder,"ir/Analysis")+file):
                    path=os.path.join(root,file)
                    shutil.move(path,os.path.join(target_folder,"ir/Analysis"))
    if root=='/mnt/e/miniC-compiler/yacc': 
        os.makedirs(os.path.join(target_folder,"yacc"),exist_ok=True)
        for file in files:
            if file.endswith(".hpp") or file.endswith(".hh"):
                if not os.path.exists(os.path.join(target_folder,"yacc")+file):
                    path=os.path.join(root,file)
                    shutil.move(path,os.path.join(target_folder,"yacc"))
    if root=='/mnt/e/miniC-compiler/lib':
        os.makedirs(os.path.join(target_folder,"lib"),exist_ok=True)
        for file in files:
            if file.endswith(".hpp"):
                if not os.path.exists(os.path.join(target_folder,"lib")+file):
                    path=os.path.join(root,file)
                    shutil.move(path,os.path.join(target_folder,"lib"))
